In [26]:
!pip install transformers
!pip install pyctcdecode
!apt-get install ffmpeg
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install jsonlines

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  Using cached https://github.com/kpu/kenlm/archive/master.zip (553 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [27]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM

import IPython.display as ipd
import torch.nn as nn
import jsonlines

In [4]:
!ffmpeg -i /content/drive/MyDrive/wav2vec2/toronto_25_278.wav -ar 16000 /content/drive/MyDrive/wav2vec2/toronto_25_278_16k_sr.wav -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [5]:
model_name = 'Yehor/wav2vec2-xls-r-base-uk-with-small-lm'
sampling_rate = 16_000
device = 'cuda' # or cpu

In [6]:
processor = Wav2Vec2Processor.from_pretrained(model_name)
# you also might init processor as the following:
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Yehor/wav2vec2-xls-r-base-uk-with-small-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [7]:
ipd.Audio('/content/drive/MyDrive/wav2vec2/toronto_25_278_16k_sr.wav')

In [8]:
audio_input, sample_rate_original = sf.read("/content/drive/MyDrive/wav2vec2/toronto_25_278_16k_sr.wav")
print(f"Original sample rate: {sample_rate_original}")

Original sample rate: 16000


In [29]:
with jsonlines.open("/content/drive/MyDrive/ukrainian-youtube-stt-dataset/dataset/labels.jsonl", 'r') as reader:
  for line in reader:
    labels = line


In [33]:
labels["dataset/toronto_25/toronto_25_278.wav"]

'А перший і єдиний кліп «У мене немає дому» – майже 10 мільйонів. Хей, люди, такі люди.'

In [9]:
input_values = processor([audio_input], sampling_rate=sampling_rate, return_tensors="pt").input_values
input_values = input_values.to(device)

In [12]:
features = processor([audio_input], sampling_rate=sampling_rate, padding=True, return_tensors='pt')
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)

with torch.no_grad():
  logits = model(input_values, attention_mask=attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(transcription)

['аа перший і єдиний кліп у мене не має дому майже десять мільйонів кілю']


In [15]:
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)

features = processor([audio_input], sampling_rate=sampling_rate, padding=True, return_tensors='pt')
input_values = features.input_values.to(device)
attention_mask = features.attention_mask.to(device)

with torch.no_grad():
  logits = model(input_values, attention_mask=attention_mask).logits

predictions = processor.batch_decode(logits.cpu().numpy()).text
print(predictions)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

['а перший і єдиний кліпу мене не має дому майже десять мільйонів к']


In [34]:
target_transcription = labels["dataset/toronto_25/toronto_25_278.wav"]


with processor.as_target_processor():
  labels = processor(target_transcription, return_tensors="pt").input_ids

loss = model(input_values, labels=labels).loss
loss.backward()

print(loss)

tensor(3.4015, device='cuda:0', grad_fn=<MeanBackward0>)


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:639: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [35]:
log_probs = nn.functional.log_softmax(logits, dim=-1, dtype=torch.float32).transpose(0, 1)

In [36]:
labels_mask = labels >= 0
target_lengths = labels_mask.sum(-1)
flattened_targets = labels.masked_select(labels_mask)

In [37]:
attention_mask = (
                attention_mask if attention_mask is not None else torch.ones_like(input_values, dtype=torch.long)
            )
input_lengths = model._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)

In [38]:
attention_mask

tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0', dtype=torch.int32)

In [39]:
ctc_loss = nn.CTCLoss()


loss = nn.functional.ctc_loss(log_probs, flattened_targets, input_lengths, target_lengths, reduction="mean", blank=36)
print(loss)

tensor(3.4015, device='cuda:0')


# Homework

In [ ]:
# TODO: implement class trainer to fine tune Wav2Vec with CTC on toronto dataset. You may refer to the HF Trainer, but you have to implement it on your own

class Trainer:
    def __init__(self, ):
        self.model = model

        self.train_dataloader = ...
        self.eval_dataloader = ...
        self.criterion = ...

        # logging, parameters, etc


    def predict(self):
        # generate prediction of the model (generate sequence)
        pass

    def train_step(self, batch):
        # calculate loss on single batch
        return loss

    def train_epoch(self, epoch, loss_metrics):
        # do backward prob, calculate WER on train epoch
        pass

    def validate(self):
        # calculate WER on validation subset
        pass

    def train(self):
        for e in range(n_epoch):
            self.train_epoch(e)
            self.validate(e - 1)


In [40]:
# Train your own Language model
# go to https://github.com/kmario23/KenLM-training

import sys
import nltk

for line in sys.stdin:
    for sentence in nltk.sent_tokenize(line):
        print(' '.join(nltk.word_tokenize(sentence)).lower())

!bzcat *your_dataset.bz2* | python preprocess.py | ./kenlm/bin/lmplz -o 3 > bible.arpa

from transformers import AutoTokenizer
from pyctcdecode import build_ctcdecoder

# do we need to remove LM?
tokenizer = AutoTokenizer.from_pretrained(model_name)

vocab_dict = tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    path_to_ngram,
)

decoder.save_to_dir("wav2vec2_with_lm")